In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
# read csv
trash_df_read = pd.read_csv('../data/trash_hauler_report_with_lat_lng.csv')
trash_df_read.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Request Number    20226 non-null  int64  
 1   Date Opened       20226 non-null  object 
 2   Request           20226 non-null  object 
 3   Description       20195 non-null  object 
 4   Incident Address  20217 non-null  object 
 5   Zip Code          20151 non-null  float64
 6   Trash Hauler      19325 non-null  object 
 7   Trash Route       19279 non-null  object 
 8   Council District  20177 non-null  float64
 9   State Plan X      20198 non-null  float64
 10  State Plan Y      20198 non-null  float64
 11  LONGITUDE         20198 non-null  float64
 12  LATITUDE          20198 non-null  float64
dtypes: float64(6), int64(1), object(6)
memory usage: 2.0+ MB


In [3]:
trash_df_read.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861


In [4]:
# find out the different request type-- There are 4 different types of Requests.
trash_df_read_requests = trash_df_read["Request"].unique().tolist()
trash_df_read_requests 

['Trash - Backdoor',
 'Trash - Curbside/Alley Missed Pickup',
 'Trash Collection Complaint',
 'Damage to Property']

In [5]:
# find out the description in different types of Requests.
#trash_df_read_requests = trash_df_read["Description"].unique().tolist()
#trash_df_read_requests 

In [6]:
#trash_df_capitalize = trash_df_read["Description"].str.capitalize()
#trash_df_capitalize

In [7]:
# find out total missed pickups based on Description Column
# trash_df_missed_total = trash_df_read[trash_df_read['Description'].str.contains('missed| miss|not pick', case=False, na=True)]
# trash_df_missed_total.info()
# trash_df_missed_total.head()


In [ ]:
# As part of Metro's contract with Red River Waste Solutions, failure to remedy an action or inaction will result in liquidated damages. One category 
# of liquidated damages is related to chronic problems in any category of service at the same premises. A chronic problem is defined as more than one 
# missed pickup for any address. The first missed pickup will not result in a fine; however, every subsequent missed pickup will result in a $200 fine.

# Your job is to determine the total amount of damages due to missed pickups. Note that not all rows that you have been provided correspond to missed
# pickups and that you will need to ensure that you are only counting missed pickups.

In [8]:
# find out total missed pickups based on Request Column
#trash_df_missed_total = trash_df_read[trash_df_read['Request']!= "Damage to Property"]

trash_df_missed_total = trash_df_read[trash_df_read["Request"].isin(["Trash - Backdoor",
 "Trash - Curbside/Alley Missed Pickup",
 "Trash Collection Complaint"])]
trash_df_missed_total.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861


In [9]:
# Assign values to the variables
missed_pickup = 1
fine_amount = 200

In [10]:
# Filter missed for RED RIVER
trash_df_missed_red_river = trash_df_missed_total[trash_df_missed_total["Trash Hauler"]=="RED RIVER"]
trash_df_missed_red_river

# Filter missed count for RED RIVER
trash_df_missed_red_river_count = trash_df_missed_red_river["Request"].value_counts()
trash_df_missed_red_river_count

Request
Trash - Curbside/Alley Missed Pickup    10715
Trash - Backdoor                         2196
Trash Collection Complaint               1287
Name: count, dtype: int64

In [11]:
# Filter on address count for Red River - Total rows 9975, 2292 rows of address >1
trash_df_missed_red_river_address_count = trash_df_missed_red_river["Incident Address"].value_counts().reset_index()
trash_df_missed_red_river_address_count = (trash_df_missed_red_river_address_count[trash_df_missed_red_river_address_count["count"] >1])
trash_df_missed_red_river_address_count

,Incident Address,count
0,3710 N NATCHEZ CT,19
1,"12546 Old Hickory Blvd, Nashville, TN 37013, U...",19
2,"6007 Obrien Ave, Nashville, TN 37209, United S...",18
3,5135 Hickory Hollow Pkwy,18
4,"802 Crescent Rd, Nashville, TN 37205, United S...",17
...,...,...
2287,"711 Greymont Dr, Nashville, TN 37217, United S...",2
2288,308 ROSEBANK AVE,2
2289,2902 brick church pike,2
2290,901 buford pl,2


In [12]:
trash_df_missed_red_river_address_count["fine_count"] = (trash_df_missed_red_river_address_count["count"] - missed_pickup).clip(lower=0)
trash_df_missed_red_river_address_count["fine_total"] = trash_df_missed_red_river_address_count["fine_count"] * fine_amount
trash_df_missed_red_river_address_count

,Incident Address,count,fine_count,fine_total
0,3710 N NATCHEZ CT,19,18,3600
1,"12546 Old Hickory Blvd, Nashville, TN 37013, U...",19,18,3600
2,"6007 Obrien Ave, Nashville, TN 37209, United S...",18,17,3400
3,5135 Hickory Hollow Pkwy,18,17,3400
4,"802 Crescent Rd, Nashville, TN 37205, United S...",17,16,3200
...,...,...,...,...
2287,"711 Greymont Dr, Nashville, TN 37217, United S...",2,1,200
2288,308 ROSEBANK AVE,2,1,200
2289,2902 brick church pike,2,1,200
2290,901 buford pl,2,1,200


In [13]:
red_river = trash_df_missed_red_river_address_count.sum()
print(red_river)

Incident Address    3710 N NATCHEZ CT12546 Old Hickory Blvd, Nashv...
count                                                            6514
fine_count                                                       4222
fine_total                                                     844400
dtype: object


In [14]:
#red_river_total_count = red_river["count"]
red_river_total_count = len(trash_df_missed_red_river)
red_river_total_count

14198

In [15]:
red_river_fine_total = red_river["fine_total"]
print("$",red_river_fine_total)

$ 844400


In [16]:
red_river_fine_count = red_river["fine_count"]
red_river_fine_count

4222

In [17]:
# find out total missed pickups based on Request Column for All Haulers
trash_df_missed_total_per_request_count = trash_df_missed_total["Request"].value_counts()
trash_df_missed_total_per_request_count

Request
Trash - Curbside/Alley Missed Pickup    15028
Trash - Backdoor                         2629
Trash Collection Complaint               2312
Name: count, dtype: int64

In [57]:
# Filter on address count for All
trash_df_missed_total_address_count = trash_df_missed_total["Incident Address"].value_counts().reset_index()
trash_df_missed_total_address_count = (trash_df_missed_total_address_count[trash_df_missed_total_address_count["count"] >1])
trash_df_missed_total_address_count

,Incident Address,count
0,5135 Hickory Hollow Pkwy,21
1,3710 N NATCHEZ CT,20
2,"12546 Old Hickory Blvd, Nashville, TN 37013, U...",19
3,"6007 Obrien Ave, Nashville, TN 37209, United S...",19
4,"802 Crescent Rd, Nashville, TN 37205, United S...",18
...,...,...
3249,1807 Lillian St,2
3250,3513 Hewlett Dr,2
3251,"6541 Cornwall Dr, Nashville, TN 37205, United ...",2
3252,3625 Woodmont Blvd,2


In [59]:
# Filter on address count for All
trash_df_missed_total_address_count["fine_count"] = (trash_df_missed_total_address_count["count"]-missed_pickup).clip(lower=0)
trash_df_missed_total_address_count["fine_total"] = trash_df_missed_total_address_count["fine_count"]*fine_amount
all_haulers = trash_df_missed_total_address_count.sum()
print(all_haulers)

Incident Address    5135 Hickory Hollow Pkwy3710 N NATCHEZ CT12546...
count                                                            9278
fine_count                                                       6024
fine_total                                                    1204800
dtype: object


In [21]:
#all_haulers_total_count = all_haulers["count"]
all_haulers_total_count = len(trash_df_missed_total)
all_haulers_total_count

19969

In [22]:
all_haulers_fine_count = all_haulers["fine_count"]
all_haulers_fine_count

6024

In [23]:
all_haulers_fine_total = all_haulers["fine_total"]
all_haulers_fine_total

1204800

In [63]:
results = pd.DataFrame({
"Hauler" : ["Red River", "All Haulers"],
"Total_count" : [red_river_total_count, all_haulers_total_count],
"Fine_Count" : [red_river_fine_count, all_haulers_fine_count],
"Fine":[red_river_fine_total, all_haulers_fine_total]
})
print(results)           

        Hauler  Total_count  Fine_Count     Fine
0    Red River        14198        4222   844400
1  All Haulers        19969        6024  1204800


In [25]:
#Part 2 - refer different notebook in the same folder:
# After determining the total amount of damages, you can look at other questions:
# ⦁	What other types of complaints are there?
# ⦁	Are there any geospatial analysis you can do?  Which visualizations can you create?
# ⦁	How do metro crews compare to the contractor's performance?
# ⦁	How much does each trash hauler owe?
# ⦁	What were to total missed pickup by route?